In [ ]:
import tensorflow as tf

def train_neural_network(mnist):

    x = tf.placeholder(tf.float32, [None, 784], name = "x-input")
    y = tf.placeholder(tf.float32, name = "y-input")

    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits= prediction, 
            labels= y),
        name= "cost"
    )
    optimizer = tf.train.AdamOptimizer().minimize(cost)

    hm_epochs = 3
    saver = tf.train.Saver()

    with tf.control_dependencies([optimizer]):
        train_op = tf.no_op(name='train')

    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples / BATCH_SIZE)):
                epoch_x, epoch_y = mnist.train.next_batch(BATCH_SIZE)
                _, c = sess.run([train_op, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME))
        saver.export_meta_graph(os.path.join(MODEL_SAVE_PATH, MODEL_NAME) + ".json", as_text=True)

# mnist dataset: https://blog.csdn.net/gaoyueace/article/details/79056085
#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
#train_neural_network(mnist)

def restore_and_retrain_model(input_checkpoint, mnist):

    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', 
                                       clear_devices=True)
    # graph = tf.get_default_graph()  # 获得默认的图
    # input_graph_def = graph.as_graph_def()  # 返回一个序列化的图代表当前的图
    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)  # 恢复图并得到数据
        graph = tf.get_default_graph()
        cost = graph.get_tensor_by_name("cost:0")
        x = graph.get_tensor_by_name("x-input:0")
        y = graph.get_tensor_by_name("y-input:0")
        prediction = graph.get_tensor_by_name("predict_output:0")
        train_op = graph.get_operation_by_name("train")
        hm_epochs = 2
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples / BATCH_SIZE)):
                epoch_x, epoch_y = mnist.train.next_batch(BATCH_SIZE)
                _, c = sess.run([train_op, cost], feed_dict={x: 
                                                                 epoch_x, 
                                                             y: 
                                                                 epoch_y})
                epoch_loss += c

            print('Epoch', epoch, 'completed out of', hm_epochs, 'loss:', epoch_loss)

        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))

        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:', accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

        saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_restored"))
        saver.export_meta_graph(os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_restored") + ".json", as_text=True)

#mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
#restore_and_retrain_model("CNN_model/cnn_model", mnist)

In [ ]:
tnd1, tnl1, tnd2, tnl2, tstd, tstl = Dataset().tri_split(0.45, 0.9)

saver = tf.train.import_meta_graph("CNN_model/cnn_model" + '.meta', 
                                   clear_devices=True)
# graph = tf.get_default_graph()  # 获得默认的图
# input_graph_def = graph.as_graph_def()  # 返回一个序列化的图代表当前的图
with tf.Session() as sess:
    saver.restore(sess, "CNN_model/cnn_model")  # 恢复图并得到数据
    
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x-input:0")
    y = graph.get_tensor_by_name("y-input:0")
    prediction = graph.get_tensor_by_name("predict_output:0")
    
    correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
    print('Accuracy:', accuracy.eval({x: tstd, y: tstl}))
    
    saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_restored"))
    saver.export_meta_graph(os.path.join(MODEL_SAVE_PATH, MODEL_NAME + "_restored") + ".json", as_text=True)
    
            
    confusion_matrix = tf.contrib.metrics.confusion_matrix(
        tf.argmax(y, 1), 
        tf.argmax(prediction, 1), 
        num_classes=10)
    print(sess.run(confusion_matrix, feed_dict= 
                                {x: tstd, 
                                 y: tstl}
             ))